In [623]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

In [479]:
def get_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    a_btn = soup.find_all('a', attrs = {'class': 'btn a-more-detail'}, href=True)
    links = []
    for result in a_btn:
        if result.text: 
            links.append("https://www.centris.ca" + result['href'])
            
    return links

def get_links_from_source(page_source):
    soup = BeautifulSoup(page_source, 'html.parser')
    a_btn = soup.find_all('a', attrs = {'class': 'btn a-more-detail'}, href=True)
    links = []
    for result in a_btn:
        if result.text: 
            links.append("https://www.centris.ca" + result['href'])
    return links
    

def get_stats(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    # gte table attributes
    keys = soup.find_all('td', attrs = {'class': ''})
    values = soup.find_all('td', attrs = {'class': 'last-child'})
    # get lat long
    coord = soup.find_all('li', attrs={'class':'onmap'})
    coords = coord[0].contents[0]['onclick'].split('q=')[-1][0:-3].split(',')
    # get image url
    image_url = soup.find_all('img', attrs={'id': 'thumbnailPhotoUrl'})[0]['src']
    image_url = image_url.replace('480', '1024')
    image_url = image_url.replace('360', '1024')
    dataset = {}
    dataset['url'] = url
    dataset['latitude'] = coords[0]
    dataset['longitude'] = coords[1]
    dataset['image_url'] = image_url
    for index, value in enumerate(values):
        dataset[keys[index].contents[0]] = value.contents[0].contents[0]
    return dataset

In [499]:
from contextlib import closing
from selenium.webdriver import PhantomJS # pip install selenium
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import requests
import lxml.html as lh
import pandas as pd
import time


all_links = []

def go_to_next_page(url, pages):
    global all_links
    # use firefox to get page with javascript generated content
    driver = PhantomJS("/Users/harman/Downloads/phantomjs-2.1.1-macosx/bin/phantomjs")
#     with open(PhantomJS("/Users/harman/Downloads/phantomjs-2.1.1-macosx/bin/phantomjs")) as driver:
    driver.get(url)
    for i in range(pages):
        button = driver.find_element_by_class_name('next')
        button.click()
        # wait for the page to load
        element = WebDriverWait(driver, 10, poll_frequency=5).until(
            EC.invisibility_of_element_located((By.ID, "deviceShowAllLink"))
        )
        # store it to string variable
        page_source = driver.page_source

        links = get_links_from_source(page_source)
#         print(len(links))
        all_links += links

In [503]:
residential_url = 'https://www.centris.ca/en/properties~for-sale~ville-marie-montreal?view=Thumbnail'
commercial_url = 'https://www.centris.ca/en/commercial-properties~for-sale~ville-marie-montreal?view=Thumbnail'

In [504]:
go_to_next_page(commercial_url, 30)

In [507]:
# url = 'https://www.centris.ca/en/commercial-properties~for-rent?uc=2&view=Thumbnail'
data = []
# urls = get_links(url)
for url in set(all_links):
    result = get_stats(url)
    data.append(result)

In [555]:
df = pd.DataFrame(data)

In [548]:
df.to_csv("centris_data.csv", index=False)

In [556]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536 entries, 0 to 535
Data columns (total 20 columns):
url                                            536 non-null object
latitude                                       536 non-null object
longitude                                      536 non-null object
image_url                                      536 non-null object
Use of property                                445 non-null object
Year built                                     309 non-null object
Available area                                 403 non-null object
Building area (at ground level)                119 non-null object
Number of units                                442 non-null object
Building style                                 33 non-null object
Lot area                                       145 non-null object
Parking                                        21 non-null object

Residential units                            42 non-null object
Main unit                   

In [557]:
df.head()

,url,latitude,longitude,image_url,Use of property,Year built,Available area,Building area (at ground level),Number of units,Building style,Lot area,Parking,\r\nResidential units,Main unit,Potential gross revenue,Zoning,Type of business,Additional features,Fireplace/Stove,\r\nResidential unit
0,https://www.centris.ca/en/commercial-buildings...,45.515628,-73.466056,https://mspublic.centris.ca/media.ashx?id=ADDD...,Commercial and office space,1972,"11,179 sqft","11,179 sqft",Commercial (1),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.centris.ca/en/4plex~for-sale~ville...,45.53494546,-73.5615702,https://mspublic.centris.ca/media.ashx?id=ADDD...,Residential only,1986,NaN,NaN,Residential (4),Attached,"2,953 sqft",Driveway (4),"3 x 4 ½, 1 x 5 ½","5 rooms, 3 bedrooms, 1 bathroom/powder room","$45,660",NaN,NaN,NaN,NaN,NaN
2,https://www.centris.ca/en/commercial-buildings...,48.649739,-72.442477,https://mspublic.centris.ca/media.ashx?id=ADDD...,Commercial and office space,NaN,"From 130 sqft to 1,194 sqft",NaN,Office (5),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.centris.ca/en/lots~for-rent~ville-...,45.49669329,-73.57020866,https://mspublic.centris.ca/media.ashx?id=ADDD...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Residential,NaN,NaN,NaN,NaN
4,https://www.centris.ca/en/commercial-buildings...,45.71283707,-73.81870081,https://mspublic.centris.ca/media.ashx?id=ADDD...,Commercial and residential,1990,"From 2,000 sqft to 13,000 sqft",NaN,Commercial (3),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [559]:
list(df["Use of property"])

In [578]:
df["Use of property"] = df["Use of property"].apply(
    lambda x: "residential" if "residential" in str(x).lower()
    else "commercial" if "commercial" in str(x).lower() 
    else "NA")

In [579]:
df.head()

,url,latitude,longitude,image_url,Use of property,Year built,Available area,Building area (at ground level),Number of units,Building style,Lot area,Parking,\r\nResidential units,Main unit,Potential gross revenue,Zoning,Type of business,Additional features,Fireplace/Stove,\r\nResidential unit
0,https://www.centris.ca/en/commercial-buildings...,45.515628,-73.466056,https://mspublic.centris.ca/media.ashx?id=ADDD...,commercial,1972,"11,179 sqft","11,179 sqft",Commercial (1),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.centris.ca/en/4plex~for-sale~ville...,45.53494546,-73.5615702,https://mspublic.centris.ca/media.ashx?id=ADDD...,residential,1986,NaN,NaN,Residential (4),Attached,"2,953 sqft",Driveway (4),"3 x 4 ½, 1 x 5 ½","5 rooms, 3 bedrooms, 1 bathroom/powder room","$45,660",NaN,NaN,NaN,NaN,NaN
2,https://www.centris.ca/en/commercial-buildings...,48.649739,-72.442477,https://mspublic.centris.ca/media.ashx?id=ADDD...,commercial,NaN,"From 130 sqft to 1,194 sqft",NaN,Office (5),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.centris.ca/en/lots~for-rent~ville-...,45.49669329,-73.57020866,https://mspublic.centris.ca/media.ashx?id=ADDD...,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Residential,NaN,NaN,NaN,NaN
4,https://www.centris.ca/en/commercial-buildings...,45.71283707,-73.81870081,https://mspublic.centris.ca/media.ashx?id=ADDD...,residential,1990,"From 2,000 sqft to 13,000 sqft",NaN,Commercial (3),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [620]:
df["Use of property"][100:115]

100             NA
101     commercial
102             NA
103     commercial
104     commercial
105    residential
106             NA
107     commercial
108             NA
109    residential
110     commercial
111     commercial
112    residential
113     commercial
114     commercial
Name: Use of property, dtype: object

In [614]:
# import time  
# import pyautogui

# time.sleep(3)  

# pyautogui.moveTo(95, 138, duration=0.5) 
# pyautogui.click()
# pyautogui.typewrite("539 Rue Sainte-Catherine Ouest, Montréal, QC", interval=0.03)
# pyautogui.typewrite(['enter'])
# time.sleep(3)

# # pyautogui.moveTo(348, 58, duration=1) 
# pyautogui.click(348, 58, duration=1, button='right')
# time.sleep(1)
# pyautogui.hotkey('escape')
# pyautogui.hotkey('command','c')
# time.sleep(1)

# pyautogui.click(777, 381, duration=1, button='right')
# time.sleep(1)
# pyautogui.hotkey('escape')
# # pyautogui.scroll(clicks=100)
# pyautogui.hotkey('command','v')
# time.sleep(2)

In [618]:
google_data_df = pd.read_csv("googledata2000.csv")

In [619]:
google_data_df.head()

,Unnamed: 0,accuracy,formatted_address,google_place_id,input_string,latitude,longitude,number_of_results,postcode,status,type
0,0,ROOFTOP,"215 Rue de la Commune O, Montréal, QC H2Y 3W8,...",ChIJRdS7aFgayUwRSmkpih97T78,"215 rue de la Commune Ouest (MTL), Montreal,C...",45.501999,-73.553966,1,H2Y 3W8,OK,street_address
1,1,ROOFTOP,"1015 Rue St-Alexandre, Montréal, QC H2Z 1N9, C...",ChIJo70z-1oayUwRrdB5G3G_8Z8,"1015 rue Saint-Alexandre (MTL), Montreal,Canada",45.503052,-73.562862,1,H2Z 1N9,OK,street_address
2,2,ROOFTOP,"221 Rue de la Commune O, Montréal, QC H2Y 2C9,...",ChIJuyKCaFgayUwRf8vYQD_y1Yo,"221 rue de la Commune Ouest (MTL), Montreal,C...",45.501829,-73.553948,1,H2Y 2C9,OK,street_address
3,3,ROOFTOP,"321 Rue de la Commune O, Montréal, QC H2Y 2E1,...",ChIJdRtkJ_YayUwRGbxEcccJjVY,"321 rue de la Commune Ouest (MTL), Montreal,C...",45.500698,-73.553653,1,H2Y 2E1,OK,street_address
4,4,ROOFTOP,"1101 Rue St-Alexandre, Montréal, QC H2Z 1P8, C...",ChIJJUFaNUUayUwRnkW4qJ5fmwU,"1101 rue Saint-Alexandre (MTL), Montreal,Canada",45.504124,-73.564819,1,H2Z 1P8,OK,street_address


In [622]:
df.head()

,url,latitude,longitude,image_url,Use of property,Year built,Available area,Building area (at ground level),Number of units,Building style,Lot area,Parking,\r\nResidential units,Main unit,Potential gross revenue,Zoning,Type of business,Additional features,Fireplace/Stove,\r\nResidential unit
0,https://www.centris.ca/en/commercial-buildings...,45.515628,-73.466056,https://mspublic.centris.ca/media.ashx?id=ADDD...,commercial,1972,"11,179 sqft","11,179 sqft",Commercial (1),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.centris.ca/en/4plex~for-sale~ville...,45.53494546,-73.5615702,https://mspublic.centris.ca/media.ashx?id=ADDD...,residential,1986,NaN,NaN,Residential (4),Attached,"2,953 sqft",Driveway (4),"3 x 4 ½, 1 x 5 ½","5 rooms, 3 bedrooms, 1 bathroom/powder room","$45,660",NaN,NaN,NaN,NaN,NaN
2,https://www.centris.ca/en/commercial-buildings...,48.649739,-72.442477,https://mspublic.centris.ca/media.ashx?id=ADDD...,commercial,NaN,"From 130 sqft to 1,194 sqft",NaN,Office (5),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.centris.ca/en/lots~for-rent~ville-...,45.49669329,-73.57020866,https://mspublic.centris.ca/media.ashx?id=ADDD...,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Residential,NaN,NaN,NaN,NaN
4,https://www.centris.ca/en/commercial-buildings...,45.71283707,-73.81870081,https://mspublic.centris.ca/media.ashx?id=ADDD...,residential,1990,"From 2,000 sqft to 13,000 sqft",NaN,Commercial (3),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [629]:
# df.drop(columns=['Building style'], inplace=True)